In [ ]:
!pip install keras-tuner

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D

from tensorflow.keras import regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform

from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import pickle

In [ ]:
%load_ext autotime
RANDOM_STATE = 42

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def view_random_image(target_dir, target_class):
    target_folder = target_dir + target_class
    random_image = random.sample(os.listdir(target_folder), 1)
    img = mpimg.imread(target_folder+'/'+random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis('off');
    #print(f"Image shape {img.shape}")
    return img

In [ ]:
import random
import matplotlib.image as mpimg
from pathlib import Path
import os.path
class_names=['buildings', 'forest', 'glacier', 'mountain' ,'sea' ,'street']
plt.figure(figsize=(20, 10))
for i in range(18):
    plt.subplot(3, 6, i+1)
    class_name = random.choice(class_names)
    img = view_random_image(target_dir='/home/abir/Downloads/greedyhpo/intel/archive/seg_train/seg_train/',target_class=class_name)

In [ ]:
train_dir = "/home/abir/Downloads/greedyhpo/intel/archive/seg_train/seg_train"
test_dir = "/home/abir/Downloads/greedyhpo/intel/archive/seg_test/seg_test"
val_dir ="/home/abir/Downloads/greedyhpo/intel/archive/seg_val/seg_test/"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale = 1./255)

training_data = train_data.flow_from_directory(train_dir,
                                              batch_size = 32,
                                              target_size = (32,32), 
                                              class_mode = 'categorical')

val_data = ImageDataGenerator(rescale = 1./255)

validation_data = val_data.flow_from_directory(val_dir,
                                            batch_size = 32,
                                            target_size = (32,32), 
                                            class_mode = 'categorical')


test_data = ImageDataGenerator(rescale = 1./255)

testing_data = test_data.flow_from_directory(test_dir,
                                            batch_size = 32,
                                            target_size = (32,32), 
                                            class_mode = 'categorical')

In [ ]:
LOG_DIR = f"{int(time.time())}"

In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def build_model(hp):
    input_shape=(32, 32, 3)
    X_input = Input(input_shape)

    X = ZeroPadding2D((1, 1))(X_input)

    X = Conv2D(hp.Choice('s1_filter_size_1', values=[16,32,64,128,256,512],default=16), (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[hp.Choice('s2_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=2, block='a', s=1)
    X = identity_block(X, 3, [hp.Choice('s2_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=2, block='b')
    X = identity_block(X, 3, [hp.Choice('s2_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s2_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=2, block='c')

    X = convolutional_block(X, f=3, filters=[hp.Choice('s3_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=3, block='a', s=2)
    X = identity_block(X, 3, [hp.Choice('s3_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=3, block='b')
    X = identity_block(X, 3, [hp.Choice('s3_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=3, block='c')
    X = identity_block(X, 3, [hp.Choice('s3_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s3_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='a', s=2)
    X = identity_block(X, 3, [hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='b')
    X = identity_block(X, 3, [hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='c')
    X = identity_block(X, 3, [hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='d')
    X = identity_block(X, 3, [hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='e')
    X = identity_block(X, 3, [hp.Choice('s4_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s4_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[hp.Choice('s5_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=5, block='a', s=2)
    X = identity_block(X, 3, [hp.Choice('s5_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=5, block='b')
    X = identity_block(X, 3, [hp.Choice('s5_filter_size_1', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_2', values=[16,32,64,128,256,512],default=16), hp.Choice('s5_filter_size_3', values=[16,32,64,128,256,512],default=128)], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    X= Dropout(hp.Choice('dp_size_1', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0))(X)
    X = Flatten()(X)
    X=Dense(hp.Choice('dense_size_1', values=[64,128,256,512],default=64), activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
    X=Dense(hp.Choice('dense_size_2', values=[64,128,256,512],default=64), activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_2', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
    X= Dropout(hp.Choice('dp_size_2', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0))(X)
    X = Dense(6,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3, 1e-4, 1e-5],default=1e-2))

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                                    optimizer=opt,
                                    metrics=['accuracy'])
    
    return model


In [ ]:
with tf.device('/device:GPU:1'):
    tuner = Hyperband(
        build_model,
        objective='val_accuracy',
        max_epochs=100,  # how many model variations to test?
        executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
        directory=LOG_DIR,
        project_name='HB_ResNet50_Intel')

    tuner.search_space_summary()
    my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]

In [ ]:
tuner.search(training_data,
             verbose=1, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=100,
             batch_size=64,
             callbacks=[my_callbacks],  # if you have callbacks like tensorboard, they go here.
             validation_data=validation_data)

In [ ]:
tuner.results_summary()
with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)

In [ ]:
new_model = tuner.get_best_hyperparameters()[0]

In [ ]:
model = tuner.hypermodel.build(new_model)

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:1'):
    hist=model.fit(training_data,
          batch_size=64,
          epochs=100,
          validation_data=validation_data,
          callbacks=[my_callbacks])

In [ ]:
_,test_acc = model.evaluate(testing_data[0][0],testing_data[0][1])